In [54]:
max1demand=6   #use 1 more than actual demand
max2demand=6   #use 1 more than actual demand
max3demand=6   #use 1 more than actual demand
max4demand=6
sparti=[]  #start with null
spartj=[]  #start with null
spartk=[]  #start with null
spartl=[]  #start with null
spartm=[]  #start with null
spartn=[]  #start with null
sidx=[]    #start with null
s=1        #start number for loop
for i in 1:8          #Create an index to call
    for j in 1:max1demand
        for k in 1:max2demand
            for l in 1:max3demand
                for m in 1:max4demand
                    for n in 1:4 #4 regions
          #statei(i+1) = s
            sparti = push!(sparti,i)  #for i in ijkl add to sparti definition
            spartj = push!(spartj,j) #for j in ijkl add to spartj definition
            spartk = push!(spartk,k) #for k in ijkl add to spartk definition
            spartl = push!(spartl,l) #for l in ijkl add to spartl definition
            spartm = push!(spartm,m) #for l in ijkl add to spartl definition
            spartn = push!(spartn,n) #for l in ijkl add to spartl definition
            sidx= push!(sidx,s)      #create index in sidx
            s=s+1
                        
                    end
                end
            end
        end
    end
end

idxidle= (sparti.<5) .& (spartj+spartk+spartl+spartm .>4)   #index all ijkl where idling cannot occur
idxregion1= (sparti.==5) .& (spartj.==1)      #index all ijkl where region 1 is active but there are no calls in region 1 to service
idxregion2= (sparti.==6) .& (spartk.==1)      #index all ijkl where region 2 is active but there are no calls in region 2 to service
idxregion3= (sparti.==7) .& (spartl.==1)      #index all ijkl where region 2 is active but there are no calls in region 3 to service
idxregion4= (sparti.==8) .& (spartm.==1)      #index all ijkl where region 2 is active but there are no calls in region 4 to service
#idxZoneFrom= (sparti.==1) .& (spartn.==(2|3|4)) .| (sparti.==2) .& (spartn.==(1|3|4)) .| (sparti.==3) .& (spartn.==(1|2|4)) .| (sparti.==4) .& (spartn.==(1|2|3))  #idx where zone from cannot occur

idxDEL=idxidle .| idxregion1 .| idxregion2 .| idxregion3 .| idxregion4 #.|idxZoneFrom #index of the previous 6 indices

deleteat!(sparti, BitArray(idxDEL)) #delete idxDEL
deleteat!(spartj, BitArray(idxDEL)) #delete idxDEL
deleteat!(spartk, BitArray(idxDEL)) #delete idxDEL
deleteat!(spartl, BitArray(idxDEL)) #delete idxDEL
deleteat!(spartm, BitArray(idxDEL)) #delete idxDEL
deleteat!(spartn, BitArray(idxDEL)) #delete idxDEL        
sidx= [1:length(sparti);]       #number of states

17296-element Vector{Int64}:
     1
     2
     3
     4
     5
     6
     7
     8
     9
    10
    11
    12
    13
     ⋮
 17285
 17286
 17287
 17288
 17289
 17290
 17291
 17292
 17293
 17294
 17295
 17296

In [20]:
μ1=5
μ2=4.5
TransferRate=   ((60/μ2)-(60/μ1))*60

80.00000000000003

In [55]:

λ=0 #Define overall Arrival Rate for every zone
N=["λ" "μ1" "μ2" "μ3" "Throughput" "Deliveries/Hour" "Avg. # in Sys" "Avg. Time in Sys" "% of Time 0 calls" "% of Time 1 call" "Compliance Rate"]

for i in 1:3
     λ= λ+.125
     μ1=5.5
  for j in 1:7
      μ1=μ1-.5
    

#Add Arrival Code

λidx1=zeros(length(sidx))          
λidx2=zeros(length(sidx)) 
λidx3=zeros(length(sidx)) 
λidx4=zeros(length(sidx)) 

μ2=.9*μ1
μ3=.85*μ1
μMatr= [μ1  μ2  μ2 μ3;     #Zone 1 to other zone rates
        μ2  μ1  μ3 μ2;    #Zone 2 to other zone rates
        μ2  μ3  μ1 μ2;     #Zone 3 to other zone rates
        μ3  μ2  μ2 μ1;]    #Zone 4 to other zone rates
μMat=zeros(length(sidx))       
matrix= zeros(length(sidx),length(sidx))


for s in 1:length(sidx)
    Zoneflag = sparti[s]
    Call1 = spartj[s]
    Call2 = spartk[s]
    Call3 = spartl[s]
    Call4 = spartm[s]
    RegionFrom= spartn[s]
    nextZoneflag = 0
    nextCall1=0
    nextCall2=0
    nextCall3=0
    nextCall4=0
    nexts=0
    nextRegionFrom=0
    RegionService= 0
    
    
 #   Arrival at region 1
    if Call1<max1demand
        nextCall1= Call1+1
        nextCall2= Call2
        nextCall3= Call3
        nextCall4= Call4
        
        if Zoneflag <=4
            nextZoneflag = 5 #don't use double equal
            nextRegionFrom=Zoneflag
        else 
            nextZoneflag= Zoneflag
            nextRegionFrom=RegionFrom            
        end      
        idx= (sparti.==nextZoneflag) .& (spartj.==nextCall1) .& (spartk.==nextCall2) .& (spartl.==nextCall3) .& (spartm.==nextCall4) .& (spartn.==nextRegionFrom)
        nexts= sidx[idx] 
        matrix[nexts,s].= matrix[nexts,s] .+ λ
        matrix[s,s]= matrix[s,s] - λ
        λidx1[s]=1
    end

    
 #   Arrival at region 2
    if Call2<max2demand
        nextCall1= Call1
        nextCall2= Call2+1
        nextCall3= Call3
        nextCall4= Call4

        if  Zoneflag <=4
            nextZoneflag = 6 #don't use double equal
            nextRegionFrom=Zoneflag
        else
            nextZoneflag= Zoneflag
            nextRegionFrom=RegionFrom
        end  

         idx= (sparti.==nextZoneflag) .& (spartj.==nextCall1) .& (spartk.==nextCall2) .& (spartl.==nextCall3) .& (spartm.==nextCall4) .& (spartn.==nextRegionFrom)
         nexts= sidx[idx] 
         matrix[nexts,s].=matrix[nexts,s] .+ λ
         matrix[s,s]= matrix[s,s] - λ
        λidx2[s]=1
    end

 #   Arrival at region 3
    if Call3<max3demand
        nextCall1= Call1
        nextCall2= Call2
        nextCall3= Call3+1
        nextCall4= Call4

        
        if Zoneflag <=4 
            nextZoneflag = 7 #don't use double equal
            nextRegionFrom=Zoneflag
        else 
            nextZoneflag= Zoneflag
            nextRegionFrom=RegionFrom
        end  
         idx= (sparti.==nextZoneflag) .& (spartj.==nextCall1) .& (spartk.==nextCall2) .& (spartl.==nextCall3) .& (spartm.==nextCall4) .& (spartn.==nextRegionFrom)
         nexts= sidx[idx] 
         matrix[nexts,s].=matrix[nexts,s].+λ
         matrix[s,s]= matrix[s,s].-λ
        λidx3[s]=1
    end  
    
 #   Arrival at region 4
    if Call4<max4demand
        nextCall1= Call1
        nextCall2= Call2
        nextCall3= Call3
        nextCall4= Call4+1

        
        if Zoneflag <=4 
            nextZoneflag = 8 #don't use double equal
            nextRegionFrom=Zoneflag
        else 
            nextZoneflag= Zoneflag
            nextRegionFrom=RegionFrom
        end  
         idx= (sparti.==nextZoneflag) .& (spartj.==nextCall1) .& (spartk.==nextCall2) .& (spartl.==nextCall3) .& (spartm.==nextCall4) .& (spartn.==nextRegionFrom)
         nexts= sidx[idx] 
         matrix[nexts,s].=matrix[nexts,s].+λ
         matrix[s,s]= matrix[s,s].-λ
        λidx4[s]=1
    end  

end   
    
# Region Transfer
TransferRate=   ((60/μ2)-(60/μ1))*60
for s in 1:length(sidx)       
    Zoneflag = sparti[s]
    Call1 = spartj[s]
    Call2 = spartk[s]
    Call3 = spartl[s]
    Call4 = spartm[s]
    RegionFrom= spartn[s]
    nextZoneflag = 0
    nextCall1=0
    nextCall2=0
    nextCall3=0
    nextCall4=0
    nexts=0
    nextRegionFrom=0
    RegionService= 0
    if Zoneflag==3
    nextZoneflag = 2 #don't use double equal
    nextRegionFrom=2
    nextCall1= Call1
    nextCall2= Call2
    nextCall3= Call3
    nextCall4= Call4
    TransferRate= 60/((60/μ3)-(60/μ1)) 
    elseif    Zoneflag==1
    nextZoneflag = 2 #don't use double equal
    nextRegionFrom=2
    nextCall1= Call1
    nextCall2= Call2
    nextCall3= Call3
    nextCall4= Call4  
    TransferRate=   60/((60/μ2)-(60/μ1))    
    elseif Zoneflag==4
    nextZoneflag = 2 #don't use double equal
    nextRegionFrom=2
    nextCall1= Call1
    nextCall2= Call2
    nextCall3= Call3
    nextCall4= Call4 
    TransferRate=   60/((60/μ2)-(60/μ1))         
    else
    nextZoneflag= Zoneflag
    nextRegionFrom= RegionFrom            
    nextCall1= Call1
    nextCall2= Call2
    nextCall3= Call3
    nextCall4= Call4
    TransferRate=0
    end
    idx= (sparti.==nextZoneflag) .& (spartj.==nextCall1) .& (spartk.==nextCall2) .& (spartl.==nextCall3) .& (spartm.==nextCall4) .& (spartn.==nextRegionFrom)
    nexts= sidx[idx]   
    matrix[nexts,s]=matrix[nexts,s].+TransferRate
    matrix[s,s]= matrix[s,s].- TransferRate        
end
#     Service from Region to Region
    
for s in 1:length(sidx)
    Zoneflag = sparti[s]
    Call1 = spartj[s]
    Call2 = spartk[s]
    Call3 = spartl[s]
    Call4 = spartm[s]
    RegionFrom= spartn[s]
    nextZoneflag = 0
    nextCall1=0
    nextCall2=0
    nextCall3=0
    nextCall4=0
    nexts=0
    nextRegionFrom=0
    RegionService= 0
 

    if Zoneflag == 5                #Formulation for Greatest Demand Policy
        if Call1-1 >=max(Call2, Call3, Call4) && Call1-1 >1
            RegionService=μMatr[1,RegionFrom]
            nextZoneflag=5
            nextCall1=Call1-1
            nextCall2=Call2
            nextCall3=Call3
            nextCall4=Call4
            nextRegionFrom=1
        elseif Call1-1 >=max(Call2, Call3, Call4) && Call1-1 ==1
            RegionService=μMatr[1,RegionFrom]
            nextZoneflag=1
            nextCall1=Call1-1
            nextCall2=Call2
            nextCall3=Call3
            nextCall4=Call4
            nextRegionFrom=1
        elseif max(Call2, Call3, Call4)==Call2
            RegionService=μMatr[1,RegionFrom]
            nextZoneflag=6
            nextCall1=Call1-1
            nextCall2=Call2
            nextCall3=Call3
            nextCall4=Call4
            nextRegionFrom=1
        elseif max(Call2, Call3, Call4)==Call3
            RegionService=μMatr[1,RegionFrom]
            nextZoneflag=7
            nextCall1=Call1-1
            nextCall2=Call2
            nextCall3=Call3
            nextCall4=Call4
            nextRegionFrom=1                        
        else
            RegionService=μMatr[1,RegionFrom]
            nextZoneflag=8
            nextCall1=Call1-1
            nextCall2=Call2
            nextCall3=Call3
            nextCall4=Call4
            nextRegionFrom=1                   
        end
 
    
    elseif Zoneflag == 6                #Formulation for Greatest Demand Policy

       if Call1 >=max(Call2-1, Call3, Call4) && Call1 >1
            RegionService=μMatr[2,RegionFrom]
            nextZoneflag=5
            nextCall1=Call1
            nextCall2=Call2-1
            nextCall3=Call3
            nextCall4=Call4
            nextRegionFrom=1
        elseif Call1 >=max(Call2-1, Call3, Call4) && Call1 ==1
            RegionService=μMatr[2,RegionFrom]
            nextZoneflag=1
            nextCall1=Call1
            nextCall2=Call2-1
            nextCall3=Call3
            nextCall4=Call4
            nextRegionFrom=1
        elseif max(Call3, Call4)<=(Call2-1)  && Call2-1 >1
            RegionService=μMatr[2,RegionFrom]
            nextZoneflag=6
            nextCall1=Call1
            nextCall2=Call2-1
            nextCall3=Call3
            nextCall4=Call4
            nextRegionFrom=1
        elseif max(Call2-1, Call3, Call4)==Call3
            RegionService=μMatr[2,RegionFrom]
            nextZoneflag=7
            nextCall1=Call1
            nextCall2=Call2-1
            nextCall3=Call3
            nextCall4=Call4
            nextRegionFrom=1   
        else
            RegionService=μMatr[2,RegionFrom]
            nextZoneflag=8
            nextCall1=Call1
            nextCall2=Call2-1
            nextCall3=Call3
            nextCall4=Call4
            nextRegionFrom=1                   
        end
 
  
    elseif Zoneflag == 7                #Formulation for Greatest Demand Policy

        if Call1 >=max(Call2, Call3-1, Call4) && Call1 >1
            RegionService=μMatr[3,RegionFrom]
            nextZoneflag=5
            nextCall1=Call1
            nextCall2=Call2
            nextCall3=Call3-1
            nextCall4=Call4
            nextRegionFrom=1
        elseif Call1 >=max(Call2, Call3-1, Call4) && Call1 ==1
            RegionService=μMatr[3,RegionFrom]
            nextZoneflag=1
            nextCall1=Call1
            nextCall2=Call2
            nextCall3=Call3-1
            nextCall4=Call4
            nextRegionFrom=1
        elseif max(Call2, Call3-1, Call4)==Call2
            RegionService=μMatr[3,RegionFrom]
            nextZoneflag=6
            nextCall1=Call1
            nextCall2=Call2
            nextCall3=Call3-1
            nextCall4=Call4
            nextRegionFrom=1
        elseif max(Call2, Call4)<=(Call3-1)
            RegionService=μMatr[3,RegionFrom]
            nextZoneflag=7
            nextCall1=Call1
            nextCall2=Call2
            nextCall3=Call3-1
            nextCall4=Call4
            nextRegionFrom=1                        
        else
            RegionService=μMatr[3,RegionFrom]
            nextZoneflag=8
            nextCall1=Call1
            nextCall2=Call2
            nextCall3=Call3-1
            nextCall4=Call4
            nextRegionFrom=1                   
        end
 
                
    elseif Zoneflag == 8                #Formulation for Greatest Demand Policy

        if Call1 >=max(Call2, Call3, Call4-1) && Call1 >1
            RegionService=μMatr[4,RegionFrom]
            nextZoneflag=5
            nextCall1=Call1
            nextCall2=Call2
            nextCall3=Call3
            nextCall4=Call4-1
            nextRegionFrom=1
        elseif Call1 >=max(Call2, Call3, Call4-1) && Call1 ==1
            RegionService=μMatr[4,RegionFrom]
            nextZoneflag=1
            nextCall1=Call1
            nextCall2=Call2
            nextCall3=Call3
            nextCall4=Call4-1
            nextRegionFrom=1
        elseif max(Call2, Call3, Call4-1)==Call2
            RegionService=μMatr[4,RegionFrom]
            nextZoneflag=6
            nextCall1=Call1
            nextCall2=Call2
            nextCall3=Call3
            nextCall4=Call4-1
            nextRegionFrom=1
        elseif max(Call2, Call3, Call4-1)==Call3
            RegionService=μMatr[4,RegionFrom]
            nextZoneflag=7
            nextCall1=Call1
            nextCall2=Call2
            nextCall3=Call3
            nextCall4=Call4-1
            nextRegionFrom=1                        
        else
            RegionService=μMatr[4,RegionFrom]
            nextZoneflag=8
            nextCall1=Call1
            nextCall2=Call2
            nextCall3=Call3
            nextCall4=Call4-1
            nextRegionFrom=1                   
        end
                     
    else
        RegionService= 0
        nextZoneflag=Zoneflag
        nextCall1=Call1
        nextCall2=Call2
        nextCall3=Call3
        nextCall4=Call4
        nextRegionFrom=RegionFrom
        #μidx=push!(μidx,0)
    end 

    idx= (sparti.==nextZoneflag) .& (spartj.==nextCall1) .& (spartk.==nextCall2) .& (spartl.==nextCall3) .& (spartm.==nextCall4) .& (spartn.==nextRegionFrom)
    nexts= sidx[idx]   
    matrix[nexts,s]=matrix[nexts,s].+RegionService
    matrix[s,s]= matrix[s,s].- RegionService 
    μMat[s,1]= μMat[s,1] .+ RegionService
 end

matrix[length(sidx),:].= 1; #prep Ahat for matrix inversion
inverse=inv(matrix) #invert Ahat matrix
sum(inverse[:,length(sidx)]) #validate that sum of π equals 1
X=inverse[:,length(sidx)] #extract π from matrix
X=round.(X,digits=10); #round to 10 digits
A=sparti[1:length(sidx)] 
B=spartj[1:length(sidx)]
C=spartk[1:length(sidx)]
D=spartl[1:length(sidx)]
E=spartm[1:length(sidx)]
F=spartn[1:length(sidx)]
M=hcat(sidx,A,B,C,D,E,F,X) 
Throughput=(λ)*sum(M[λidx1.==1,8])+(λ)*sum(M[λidx2.==1,8])+(λ)*sum(M[λidx3.==1,8])+(λ)*sum(M[λidx4.==1,8])
Gain=μMat'*X
L=0
for s in 1:length(sidx)
  L=L+(B[s]+C[s]+D[s]+E[s]-4)*X[s]
end
W=L/Throughput 
Midx0= (spartj .+ spartk .+ spartl.+spartm).==4
ZeroCall=sum(M[Midx0.==1,8])
#Percentage of time with one calls
Midx1= (spartj .+ spartk .+ spartl.+spartm).==5
OneCall=sum(M[Midx1.==1,8])
CR=ZeroCall+OneCall
N=[N; λ μ1 μ2 μ3 Throughput Gain L W ZeroCall OneCall CR]
end
 end

In [53]:
using DataFrames
df= DataFrame(N[2:Int(length(N)/11),:], :auto)


,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10
,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any
1,0.125,5.0,4.5,4.25,0.499566,0.499566,0.123205,0.246624,0.890128,0.0978842
2,0.125,4.5,4.05,3.825,0.499456,0.499456,0.138689,0.277681,0.877947,0.107276
3,0.125,4.0,3.6,3.4,0.499299,0.499299,0.158604,0.317653,0.862734,0.118601
4,0.125,3.5,3.15,2.975,0.499062,0.499062,0.185153,0.371002,0.843199,0.132486
5,0.125,3.0,2.7,2.55,0.498683,0.498683,0.22228,0.445734,0.817205,0.149823
6,0.125,2.5,2.25,2.125,0.498019,0.498019,0.277774,0.557759,0.78094,0.171849
7,0.125,2.0,1.8,1.7,0.496697,0.496697,0.369347,0.743608,0.726906,0.200035
8,0.25,5.0,4.5,4.25,0.996038,0.996038,0.277774,0.278879,0.78094,0.171849
9,0.25,4.5,4.05,3.825,0.994965,0.994965,0.31719,0.318795,0.756863,0.185091


In [50]:
using CSV
CSV.write("C:\\Users\\alexb\\Documents\\NCSU\\501\\Thesis\\4ZoneFAdjZGD_AllArrival_AmbStats9FEB_1.csv",df)

"C:\\Users\\alexb\\Documents\\NCSU\\501\\Thesis\\4ZoneFAdjZGD_AllArrival_AmbStats9FEB_1.csv"

In [27]:
λ=.125
μ1=5

#Add Arrival Code

λidx1=zeros(length(sidx))          
λidx2=zeros(length(sidx)) 
λidx3=zeros(length(sidx)) 
λidx4=zeros(length(sidx)) 

μ2=.9*μ1
μ3=.85*μ1
μMatr= [μ1  μ2  μ2 μ3;     #Zone 1 to other zone rates
        μ2  μ1  μ3 μ2;    #Zone 2 to other zone rates
        μ2  μ3  μ1 μ2;     #Zone 3 to other zone rates
        μ3  μ2  μ2 μ1;]    #Zone 4 to other zone rates
μMat=zeros(length(sidx))       
matrix= zeros(length(sidx),length(sidx))


for s in 1:length(sidx)
    Zoneflag = sparti[s]
    Call1 = spartj[s]
    Call2 = spartk[s]
    Call3 = spartl[s]
    Call4 = spartm[s]
    RegionFrom= spartn[s]
    nextZoneflag = 0
    nextCall1=0
    nextCall2=0
    nextCall3=0
    nextCall4=0
    nexts=0
    nextRegionFrom=0
    RegionService= 0
    
    
 #   Arrival at region 1
    if Call1<max1demand
        nextCall1= Call1+1
        nextCall2= Call2
        nextCall3= Call3
        nextCall4= Call4
        
        if Zoneflag <=4
            nextZoneflag = 5 #don't use double equal
            nextRegionFrom=Zoneflag
        else 
            nextZoneflag= Zoneflag
            nextRegionFrom=RegionFrom            
        end      
        idx= (sparti.==nextZoneflag) .& (spartj.==nextCall1) .& (spartk.==nextCall2) .& (spartl.==nextCall3) .& (spartm.==nextCall4) .& (spartn.==nextRegionFrom)
        nexts= sidx[idx] 
        matrix[nexts,s].= matrix[nexts,s] .+ λ
        matrix[s,s]= matrix[s,s] - λ
        λidx1[s]=1
    end

    
 #   Arrival at region 2
    if Call2<max2demand
        nextCall1= Call1
        nextCall2= Call2+1
        nextCall3= Call3
        nextCall4= Call4

        if  Zoneflag <=4
            nextZoneflag = 6 #don't use double equal
            nextRegionFrom=Zoneflag
        else
            nextZoneflag= Zoneflag
            nextRegionFrom=RegionFrom
        end  

         idx= (sparti.==nextZoneflag) .& (spartj.==nextCall1) .& (spartk.==nextCall2) .& (spartl.==nextCall3) .& (spartm.==nextCall4) .& (spartn.==nextRegionFrom)
         nexts= sidx[idx] 
         matrix[nexts,s].=matrix[nexts,s] .+ λ
         matrix[s,s]= matrix[s,s] - λ
        λidx2[s]=1
    end

 #   Arrival at region 3
    if Call3<max3demand
        nextCall1= Call1
        nextCall2= Call2
        nextCall3= Call3+1
        nextCall4= Call4

        
        if Zoneflag <=4 
            nextZoneflag = 7 #don't use double equal
            nextRegionFrom=Zoneflag
        else 
            nextZoneflag= Zoneflag
            nextRegionFrom=RegionFrom
        end  
         idx= (sparti.==nextZoneflag) .& (spartj.==nextCall1) .& (spartk.==nextCall2) .& (spartl.==nextCall3) .& (spartm.==nextCall4) .& (spartn.==nextRegionFrom)
         nexts= sidx[idx] 
         matrix[nexts,s].=matrix[nexts,s].+λ
         matrix[s,s]= matrix[s,s].-λ
        λidx3[s]=1
    end  
    
 #   Arrival at region 4
    if Call4<max4demand
        nextCall1= Call1
        nextCall2= Call2
        nextCall3= Call3
        nextCall4= Call4+1

        
        if Zoneflag <=4 
            nextZoneflag = 8 #don't use double equal
            nextRegionFrom=Zoneflag
        else 
            nextZoneflag= Zoneflag
            nextRegionFrom=RegionFrom
        end  
         idx= (sparti.==nextZoneflag) .& (spartj.==nextCall1) .& (spartk.==nextCall2) .& (spartl.==nextCall3) .& (spartm.==nextCall4) .& (spartn.==nextRegionFrom)
         nexts= sidx[idx] 
         matrix[nexts,s].=matrix[nexts,s].+λ
         matrix[s,s]= matrix[s,s].-λ
        λidx4[s]=1
    end  

end   
    
# Region Transfer
TransferRate=   ((60/μ2)-(60/μ1))*60
for s in 1:length(sidx)       
    Zoneflag = sparti[s]
    Call1 = spartj[s]
    Call2 = spartk[s]
    Call3 = spartl[s]
    Call4 = spartm[s]
    RegionFrom= spartn[s]
    nextZoneflag = 0
    nextCall1=0
    nextCall2=0
    nextCall3=0
    nextCall4=0
    nexts=0
    nextRegionFrom=0
    RegionService= 0
    if Zoneflag==1
    nextZoneflag = 4 #don't use double equal
    nextRegionFrom=4
    nextCall1= Call1
    nextCall2= Call2
    nextCall3= Call3
    nextCall4= Call4
    TransferRate= 60/((60/μ3)-(60/μ1)) 
    elseif    Zoneflag==2
    nextZoneflag = 4 #don't use double equal
    nextRegionFrom=4
    nextCall1= Call1
    nextCall2= Call2
    nextCall3= Call3
    nextCall4= Call4  
    TransferRate=   60/((60/μ2)-(60/μ1))    
    elseif Zoneflag==3
    nextZoneflag = 4 #don't use double equal
    nextRegionFrom=4
    nextCall1= Call1
    nextCall2= Call2
    nextCall3= Call3
    nextCall4= Call4 
    TransferRate=   60/((60/μ2)-(60/μ1))         
    else
    nextZoneflag= Zoneflag
    nextRegionFrom= RegionFrom            
    nextCall1= Call1
    nextCall2= Call2
    nextCall3= Call3
    nextCall4= Call4
    TransferRate=0
    end
    idx= (sparti.==nextZoneflag) .& (spartj.==nextCall1) .& (spartk.==nextCall2) .& (spartl.==nextCall3) .& (spartm.==nextCall4) .& (spartn.==nextRegionFrom)
    nexts= sidx[idx]   
    matrix[nexts,s]=matrix[nexts,s].+TransferRate
    matrix[s,s]= matrix[s,s].- TransferRate        
end
#     Service from Region to Region
    
for s in 1:length(sidx)
    Zoneflag = sparti[s]
    Call1 = spartj[s]
    Call2 = spartk[s]
    Call3 = spartl[s]
    Call4 = spartm[s]
    RegionFrom= spartn[s]
    nextZoneflag = 0
    nextCall1=0
    nextCall2=0
    nextCall3=0
    nextCall4=0
    nexts=0
    nextRegionFrom=0
    RegionService= 0
 

    if Zoneflag == 5                #Formulation for Greatest Demand Policy
        if Call1-1 >=max(Call2, Call3, Call4) && Call1-1 >1
            RegionService=μMatr[1,RegionFrom]
            nextZoneflag=5
            nextCall1=Call1-1
            nextCall2=Call2
            nextCall3=Call3
            nextCall4=Call4
            nextRegionFrom=1
        elseif Call1-1 >=max(Call2, Call3, Call4) && Call1-1 ==1
            RegionService=μMatr[1,RegionFrom]
            nextZoneflag=1
            nextCall1=Call1-1
            nextCall2=Call2
            nextCall3=Call3
            nextCall4=Call4
            nextRegionFrom=1
        elseif max(Call2, Call3, Call4)==Call2
            RegionService=μMatr[1,RegionFrom]
            nextZoneflag=6
            nextCall1=Call1-1
            nextCall2=Call2
            nextCall3=Call3
            nextCall4=Call4
            nextRegionFrom=1
        elseif max(Call2, Call3, Call4)==Call3
            RegionService=μMatr[1,RegionFrom]
            nextZoneflag=7
            nextCall1=Call1-1
            nextCall2=Call2
            nextCall3=Call3
            nextCall4=Call4
            nextRegionFrom=1                        
        else
            RegionService=μMatr[1,RegionFrom]
            nextZoneflag=8
            nextCall1=Call1-1
            nextCall2=Call2
            nextCall3=Call3
            nextCall4=Call4
            nextRegionFrom=1                   
        end
 
    
    elseif Zoneflag == 6                #Formulation for Greatest Demand Policy

       if Call1 >=max(Call2-1, Call3, Call4) && Call1 >1
            RegionService=μMatr[2,RegionFrom]
            nextZoneflag=5
            nextCall1=Call1
            nextCall2=Call2-1
            nextCall3=Call3
            nextCall4=Call4
            nextRegionFrom=1
        elseif Call1 >=max(Call2-1, Call3, Call4) && Call1 ==1
            RegionService=μMatr[2,RegionFrom]
            nextZoneflag=1
            nextCall1=Call1
            nextCall2=Call2-1
            nextCall3=Call3
            nextCall4=Call4
            nextRegionFrom=1
        elseif max(Call3, Call4)<=(Call2-1)  && Call2-1 >1
            RegionService=μMatr[2,RegionFrom]
            nextZoneflag=6
            nextCall1=Call1
            nextCall2=Call2-1
            nextCall3=Call3
            nextCall4=Call4
            nextRegionFrom=1
        elseif max(Call2-1, Call3, Call4)==Call3
            RegionService=μMatr[2,RegionFrom]
            nextZoneflag=7
            nextCall1=Call1
            nextCall2=Call2-1
            nextCall3=Call3
            nextCall4=Call4
            nextRegionFrom=1   
        else
            RegionService=μMatr[2,RegionFrom]
            nextZoneflag=8
            nextCall1=Call1
            nextCall2=Call2-1
            nextCall3=Call3
            nextCall4=Call4
            nextRegionFrom=1                   
        end
 
  
    elseif Zoneflag == 7                #Formulation for Greatest Demand Policy

        if Call1 >=max(Call2, Call3-1, Call4) && Call1 >1
            RegionService=μMatr[3,RegionFrom]
            nextZoneflag=5
            nextCall1=Call1
            nextCall2=Call2
            nextCall3=Call3-1
            nextCall4=Call4
            nextRegionFrom=1
        elseif Call1 >=max(Call2, Call3-1, Call4) && Call1 ==1
            RegionService=μMatr[3,RegionFrom]
            nextZoneflag=1
            nextCall1=Call1
            nextCall2=Call2
            nextCall3=Call3-1
            nextCall4=Call4
            nextRegionFrom=1
        elseif max(Call2, Call3-1, Call4)==Call2
            RegionService=μMatr[3,RegionFrom]
            nextZoneflag=6
            nextCall1=Call1
            nextCall2=Call2
            nextCall3=Call3-1
            nextCall4=Call4
            nextRegionFrom=1
        elseif max(Call2, Call4)<=(Call3-1)
            RegionService=μMatr[3,RegionFrom]
            nextZoneflag=7
            nextCall1=Call1
            nextCall2=Call2
            nextCall3=Call3-1
            nextCall4=Call4
            nextRegionFrom=1                        
        else
            RegionService=μMatr[3,RegionFrom]
            nextZoneflag=8
            nextCall1=Call1
            nextCall2=Call2
            nextCall3=Call3-1
            nextCall4=Call4
            nextRegionFrom=1                   
        end
 
                
    elseif Zoneflag == 8                #Formulation for Greatest Demand Policy

        if Call1 >=max(Call2, Call3, Call4-1) && Call1 >1
            RegionService=μMatr[4,RegionFrom]
            nextZoneflag=5
            nextCall1=Call1
            nextCall2=Call2
            nextCall3=Call3
            nextCall4=Call4-1
            nextRegionFrom=1
        elseif Call1 >=max(Call2, Call3, Call4-1) && Call1 ==1
            RegionService=μMatr[4,RegionFrom]
            nextZoneflag=1
            nextCall1=Call1
            nextCall2=Call2
            nextCall3=Call3
            nextCall4=Call4-1
            nextRegionFrom=1
        elseif max(Call2, Call3, Call4-1)==Call2
            RegionService=μMatr[4,RegionFrom]
            nextZoneflag=6
            nextCall1=Call1
            nextCall2=Call2
            nextCall3=Call3
            nextCall4=Call4-1
            nextRegionFrom=1
        elseif max(Call2, Call3, Call4-1)==Call3
            RegionService=μMatr[4,RegionFrom]
            nextZoneflag=7
            nextCall1=Call1
            nextCall2=Call2
            nextCall3=Call3
            nextCall4=Call4-1
            nextRegionFrom=1                        
        else
            RegionService=μMatr[4,RegionFrom]
            nextZoneflag=8
            nextCall1=Call1
            nextCall2=Call2
            nextCall3=Call3
            nextCall4=Call4-1
            nextRegionFrom=1                   
        end
                     
    else
        RegionService= 0
        nextZoneflag=Zoneflag
        nextCall1=Call1
        nextCall2=Call2
        nextCall3=Call3
        nextCall4=Call4
        nextRegionFrom=RegionFrom
        #μidx=push!(μidx,0)
    end 

    idx= (sparti.==nextZoneflag) .& (spartj.==nextCall1) .& (spartk.==nextCall2) .& (spartl.==nextCall3) .& (spartm.==nextCall4) .& (spartn.==nextRegionFrom)
    nexts= sidx[idx]   
    matrix[nexts,s]=matrix[nexts,s].+RegionService
    matrix[s,s]= matrix[s,s].- RegionService 
    μMat[s,1]= μMat[s,1] .+ RegionService
 end

In [29]:
sum(matrix)

0.0

In [30]:
for s in 1:length(sidx)
    su=sum(matrix[:,s])
    if su!=0
        println(s)
        println(su)
    end
end
    

In [31]:
matrix[length(sidx),:].= 1; #prep Ahat for matrix inversion
inverse=inv(matrix) #invert Ahat matrix
sum(inverse[:,length(sidx)]) #validate that sum of π equals 1
X=inverse[:,length(sidx)] #extract π from matrix
X=round.(X,digits=10); #round to 10 digits
A=sparti[1:length(sidx)] 
B=spartj[1:length(sidx)]
C=spartk[1:length(sidx)]
D=spartl[1:length(sidx)]
E=spartm[1:length(sidx)]
F=spartn[1:length(sidx)]
M=hcat(sidx,A,B,C,D,E,F,X) 
Throughput=(λ)*sum(M[λidx1.==1,8])+(λ)*sum(M[λidx2.==1,8])+(λ)*sum(M[λidx3.==1,8])+(λ)*sum(M[λidx4.==1,8])
Gain=μMat'*X
L=0
for s in 1:length(sidx)
  L=L+(B[s]+C[s]+D[s]+E[s]-4)*X[s]
end
W=L/Throughput 
Midx0= (spartj .+ spartk .+ spartl.+spartm).==4
ZeroCall=sum(M[Midx0.==1,8])
#Percentage of time with one calls
Midx1= (spartj .+ spartk).==3
OneCall=sum(M[Midx1.==1,8])
CR=ZeroCall+OneCall
N=[N; λ μ1 μ2 μ3 Throughput Gain L W ZeroCall OneCall CR]

12×11 Matrix{Any}:
  "λ"    "μ1"   "μ2"   "μ3"   "Throughput"  …   "Compliance Rate"
 0.125  5.0    4.5    4.25   0.499566          0.946016
 0.125  4.5    4.05   3.825  0.499456          0.939853
 0.125  4.0    3.6    3.4    0.499299          0.932099
 0.125  3.5    3.15   2.975  0.499062          0.922047
 0.125  3.0    2.7    2.55   0.498683       …  0.908499
 0.125  2.5    2.25   2.125  0.498019          0.889256
 0.125  2.0    1.8    1.7    0.496696          0.859801
 0.25   1.5    1.35   1.275  0.924421          0.597495
 0.25   1.0    0.9    0.85   0.799173          0.40049
 0.25   0.5    0.45   0.425  0.457801       …  0.0985647
 0.125  5      4.5    4.25   0.499566          0.946016

In [56]:
using DataFrames
df= DataFrame(N[2:Int(length(N)/11),:], :auto)

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10
,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any
1,0.125,5.0,4.5,4.25,0.5,0.5,0.123556,0.247112,0.890033,0.0978739
2,0.125,4.5,4.05,3.825,0.5,0.5,0.13919,0.27838,0.877814,0.10726
3,0.125,4.0,3.6,3.4,0.5,0.5,0.159352,0.318703,0.862541,0.118575
4,0.125,3.5,3.15,2.975,0.5,0.5,0.186341,0.372683,0.842904,0.13244
5,0.125,3.0,2.7,2.55,0.5,0.5,0.224333,0.448665,0.816721,0.149736
6,0.125,2.5,2.25,2.125,0.5,0.499999,0.281772,0.563544,0.780066,0.17166
7,0.125,2.0,1.8,1.7,0.499998,0.499998,0.378725,0.757453,0.725083,0.19954
8,0.25,5.0,4.5,4.25,0.999999,0.999999,0.281772,0.281772,0.780066,0.17166
9,0.25,4.5,4.05,3.825,0.999998,0.999998,0.323135,0.323135,0.755629,0.184792


In [57]:
using CSV
CSV.write("C:\\Users\\alexb\\Documents\\NCSU\\501\\Thesis\\4ZoneFADJFZ_AllArrival_AmbStats10FEB_1.csv",df)

"C:\\Users\\alexb\\Documents\\NCSU\\501\\Thesis\\4ZoneFADJFZ_AllArrival_AmbStats10FEB_1.csv"